In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns',40)
pd.set_option('display.precision',3)

In [40]:
def calc_PA(x):
    return x.AB+x.BB+x.HBP+x.SF+x.SH

def calc_TB(x):
    return x.H+x['2B']+(x['3B'])*2+(x.HR)*3

def calc_AVG(x):
    if x.AB==0:
        return 0
    else:
        return x.H/x.AB

def calc_OBP(x):
    if (x.AB+x.BB+x.HBP+x.SF)==0:
        return 0
    else:
        return (x.H+x.BB+x.HBP)/(x.AB+x.BB+x.HBP+x.SF)

def calc_SLG(x):
    if x.AB==0:
        return 0
    else:
        return x.TB/x.AB  

def calc_OPS(x):
    return x.OBP+x.SLG

def calc_ISO(x):
    return x.SLG-x.AVG

def calc_BABIP(x):
    if (x.AB-x.SO-x.HR+x.SF)==0:
        return 0
    else:
        return (x.H-x.HR)/(x.AB-x.SO-x.HR+x.SF)

def calc_wOBA(x):
    if (x.AB+x.BB-x.IBB+x.SF+x.HBP)==0:
        return 0
    else:
        return ((x.BB-x.IBB)*x.wBB+x.HBP*x.wHBP+(x.H-x['2B']-x['3B']-x.HR)*x.w1B+x['2B']*x.w2B+x['3B']*x.w3B+x.HR*x.wHR)/(x.AB+x.BB-x.IBB+x.SF+x.HBP)

def calc_RC(x):
    if (x.AB+x.BB)==0:
        return 0
    else:
        return (x.TB*(x.H+x.BB)/(x.AB+x.BB))

def calc_wRC(x):
    return (((x.wOBA-x.lgwOBA)/x.wOBAScale)+(x['lgR/PA']))*x.PA

def calc_wRAA(x):
    if x.PA==0:
        return 0
    else:
        return ((x.wOBA-x.lgwOBA)/x.wOBAScale)*x.PA

def calc_WHIP(x):
    if x.IPouts==0:
        return 0
    else:
        return (x.BB+x.H)/x.IPouts*3

def calc_FIP(x):
    if x.IPouts==0:
        return 0
    else:
        return (((13*x.HR)+(3*(x.BB+x.HBP))-(2*x.SO))/x.IPouts*3) + x.cFIP

def calc_K9(x):
    if x.IPouts==0:
        return 0
    else:
        return 9*x.SO/x.IPouts*3

def calc_BB9(x):
    if x.IPouts==0:
        return 0
    else:
        return 9*x.BB/x.IPouts*3

def calc_KBB(x):
    if x.BB == 0:
        return 0
    else:
        return x.SO/x.BB



In [26]:
df_constants=pd.read_csv('../data/raw/fangraphs_constants.csv')
df_constants.rename(columns={"wOBA":"lgwOBA","R/PA":"lgR/PA"},inplace=True)
df_constants.head()

,Season,lgwOBA,wOBAScale,wBB,wHBP,w1B,w2B,w3B,wHR,runSB,runCS,lgR/PA,R/W,cFIP
0,2022,0.307,1.287,0.692,0.725,0.892,1.278,1.626,2.109,0.2,-0.393,0.113,9.445,3.125
1,2021,0.314,1.209,0.692,0.722,0.879,1.242,1.568,2.007,0.2,-0.419,0.121,9.973,3.170
2,2020,0.320,1.185,0.699,0.728,0.883,1.238,1.558,1.979,0.2,-0.435,0.126,10.282,3.191
3,2019,0.320,1.157,0.690,0.719,0.870,1.217,1.529,1.940,0.2,-0.435,0.126,10.296,3.214
4,2018,0.315,1.226,0.690,0.720,0.880,1.247,1.578,2.031,0.2,-0.407,0.117,9.714,3.161


In [41]:
df_bat=pd.read_csv('../data/raw/Batting.csv')
df_bat=df_bat[df_bat['yearID']>2000]
df_bat=pd.merge(df_bat,df_constants[['Season','lgwOBA','wOBAScale','wBB','wHBP','w1B','w2B','w3B','wHR','lgR/PA']],how='left',left_on='yearID',right_on='Season')
df_bat['PA']=df_bat.apply(calc_PA,axis=1)
df_bat['TB']=df_bat.apply(calc_TB,axis=1)
df_bat['AVG']=df_bat.apply(calc_AVG,axis=1)
df_bat['OBP']=df_bat.apply(calc_OBP,axis=1)
df_bat['SLG']=df_bat.apply(calc_SLG,axis=1)
df_bat['OPS']=df_bat.apply(calc_OPS,axis=1)
df_bat['ISO']=df_bat.apply(calc_ISO,axis=1)
df_bat['BABIP']=df_bat.apply(calc_BABIP,axis=1)
df_bat['RC']=df_bat.apply(calc_RC,axis=1)
df_bat['wOBA']=df_bat.apply(calc_wOBA,axis=1)
df_bat['wRAA']=df_bat.apply(calc_wRAA,axis=1)
df_bat['wRC']=df_bat.apply(calc_wRC,axis=1)
df_bat.tail()

,playerID,yearID,stint,teamID,lgID,G,AB,R,H,2B,3B,HR,RBI,SB,CS,BB,SO,IBB,HBP,SH,...,wOBAScale,wBB,wHBP,w1B,w2B,w3B,wHR,lgR/PA,PA,TB,AVG,OBP,SLG,OPS,ISO,BABIP,RC,wOBA,wRAA,wRC
29841,zimmejo02,2021,1,MIL,NL,2,1,0,0,0,0,0,0.0,0.0,0.0,0,1.0,0.0,0.0,0.0,...,1.209,0.692,0.722,0.879,1.242,1.568,2.007,0.121,1.0,0,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,-0.260,-0.139
29842,zimmeky01,2021,1,KCA,AL,52,0,0,0,0,0,0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,...,1.209,0.692,0.722,0.879,1.242,1.568,2.007,0.121,0.0,0,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,-0.000
29843,zimmery01,2021,1,WAS,NL,110,255,27,62,16,0,14,46.0,0.0,0.0,16,77.0,0.0,0.0,0.0,...,1.209,0.692,0.722,0.879,1.242,1.568,2.007,0.121,273.0,120,0.243,0.286,0.471,0.756,0.227,0.289,34.539,0.319,1.198,34.231
29844,zuberty01,2021,1,KCA,AL,31,1,0,0,0,0,0,0.0,0.0,0.0,0,1.0,0.0,0.0,0.0,...,1.209,0.692,0.722,0.879,1.242,1.568,2.007,0.121,1.0,0,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,-0.260,-0.139
29845,zuninmi01,2021,1,TBA,AL,109,333,64,72,11,2,33,62.0,0.0,0.0,34,132.0,0.0,7.0,0.0,...,1.209,0.692,0.722,0.879,1.242,1.568,2.007,0.121,375.0,186,0.216,0.301,0.559,0.860,0.342,0.231,53.722,0.359,13.825,59.200


In [46]:
df_pit=pd.read_csv('../data/raw/Pitching.csv')
df_pit=pd.merge(df_pit,df_constants[['Season','cFIP']],how='left',left_on='yearID',right_on='Season')
df_pit['WHIP']=df_pit.apply(calc_WHIP,axis=1)
df_pit['FIP']=df_pit.apply(calc_FIP,axis=1)
df_pit['SO/BB']=df_pit.apply(calc_KBB,axis=1)
df_pit['SO9']=df_pit.apply(calc_K9,axis=1)
df_pit.tail(10)

,playerID,yearID,stint,teamID,lgID,W,L,G,GS,CG,SHO,SV,IPouts,H,ER,HR,BB,SO,BAOpp,ERA,IBB,WP,HBP,BK,BFP,GF,R,SH,SF,GIDP,Season,cFIP,WHIP,FIP,SO/BB,SO9
49420,ynoahu01,2021,1,ATL,NL,4,6,18,17,0,0,0,273,76,41,14,25,100,0.224,4.05,1.0,4,4.0,0,372.0,0,42,1.0,2.0,5.0,2021,3.17,1.110,3.928,4.000,9.890
49421,youngal01,2021,1,ARI,NL,2,6,30,2,0,0,0,125,50,29,11,20,38,0.298,6.26,3.0,2,1.0,0,193.0,7,34,2.0,2.0,4.0,2021,3.17,1.680,6.290,1.900,8.208
49422,youngal01,2021,2,CLE,AL,0,0,10,0,0,0,0,31,15,9,1,7,5,0.341,7.84,1.0,1,2.0,0,53.0,5,9,0.0,0.0,3.0,2021,3.17,2.129,6.073,0.714,4.355
49423,zamorda01,2021,1,SEA,AL,2,0,4,0,0,0,0,13,5,3,1,1,3,0.294,6.23,0.0,0,0.0,0,18.0,1,4,0.0,0.0,1.0,2021,3.17,1.385,5.478,3.000,6.231
49424,zerpaan01,2021,1,KCA,AL,0,1,1,1,0,0,0,15,3,0,0,1,4,0.176,0.00,0.0,0,0.0,0,20.0,0,2,1.0,1.0,0.0,2021,3.17,0.800,2.170,4.000,7.200
49425,zeuchtj01,2021,1,TOR,AL,0,2,5,3,0,0,0,45,21,11,6,9,8,0.323,6.60,0.0,0,0.0,0,74.0,0,16,0.0,0.0,2.0,2021,3.17,2.000,9.103,0.889,4.800
49426,zimmebr02,2021,1,BAL,AL,4,5,14,13,0,0,0,193,75,36,14,22,56,0.291,5.04,0.0,0,2.0,1,285.0,0,37,1.0,2.0,3.0,2021,3.17,1.508,5.377,2.545,7.834
49427,zimmejo02,2021,1,MIL,NL,0,0,2,0,0,0,0,17,8,5,1,2,0,0.348,7.94,0.0,0,1.0,0,26.0,1,5,0.0,0.0,1.0,2021,3.17,1.765,7.052,0.000,0.000
49428,zimmeky01,2021,1,KCA,AL,4,1,52,2,0,0,2,162,46,29,7,30,46,0.243,4.83,1.0,9,0.0,0,223.0,9,32,0.0,4.0,13.0,2021,3.17,1.407,4.818,1.533,7.667
49429,zuberty01,2021,1,KCA,AL,0,3,31,0,0,0,0,82,26,19,6,17,25,0.250,6.26,1.0,2,1.0,0,123.0,6,20,0.0,1.0,2.0,2021,3.17,1.573,6.170,1.471,8.232


In [43]:
df_pit[df_pit['playerID'].str.contains('ohtan')]

,playerID,yearID,stint,teamID,lgID,W,L,G,GS,CG,SHO,SV,IPouts,H,ER,HR,BB,SO,BAOpp,ERA,IBB,WP,HBP,BK,BFP,GF,R,SH,SF,GIDP,Season,cFIP,WHIP,FIP,SO/BB,SO9
46395,ohtansh01,2018,1,LAA,AL,4,2,10,10,0,0,0,155,38,19,6,22,63,0.203,3.31,0.0,5,1.0,0,211.0,0,19,0.0,1.0,2.0,2018,3.161,1.161,3.567,2.864,10.974
48137,ohtansh01,2020,1,LAA,AL,0,1,2,2,0,0,0,5,3,7,0,8,3,0.375,37.80,0.0,1,0.0,0,16.0,0,7,0.0,0.0,0.0,2020,3.191,6.600,13.991,0.375,16.200
49067,ohtansh01,2021,1,LAA,AL,9,2,23,23,0,0,0,391,98,46,15,44,156,0.207,3.18,2.0,10,10.0,2,533.0,0,48,2.0,4.0,8.0,2021,3.170,1.090,3.515,3.545,10.772
